In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import timm
from torchmetrics import Metric



In [2]:

class NMAE(Metric):
    def __init__(self, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.add_state("nmae", default=torch.tensor(0.), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")
    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        nmae = torch.sum(torch.mean(torch.abs(target-preds),dim=-1) / torch.mean(torch.abs(target), dim=-1))
        n_obs = target.numel()
        self.nmae += nmae
        self.total += n_obs
    def compute(self):
        return self.nmae / self.total


In [3]:
preprocessing_train_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TRAIN_224/*")
preprocessing_test_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TEST_224/*")

train_path = []
train_imgs = []
for path in tqdm(preprocessing_train_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    train_path.append(path)
    train_imgs.append(img)

test_path = []
test_imgs = []
for path in tqdm(preprocessing_test_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    test_path.append(path)
    test_imgs.append(img)

train_df = pd.read_parquet("D:/DACON_GROWTH/train.parquet")
test_df = pd.read_parquet("D:/DACON_GROWTH/test.parquet")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 460/460 [00:02<00:00, 166.51it/s]


In [4]:
img2weight = {}
for i in tqdm(train_df["이미지"].unique()):
    temp = train_df[train_df["이미지"]==i]
    weight = temp["무게"].values[0]
    img2weight[i] = weight

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1592/1592 [04:54<00:00,  5.41it/s]


In [5]:

train_df["시간2"] = train_df["시간"].apply(lambda x: x.split(" ")[-1])
golden_time = ['17:40', '17:41', '17:42', '17:43', '17:44', '17:45', '17:46', '17:47', '17:48', '17:49', '17:50',
               '17:51', '17:52', '17:53', '17:54', '17:55', '17:56', '17:57', '17:58', '17:59', '18:00', '18:01',
               '18:02', '18:03', '18:04', '18:05']
golden_dict ={}
for e, i in enumerate(train_df["시간2"].unique()):
       golden_dict[i] = e
train_df["순서"] = train_df["시간2"].apply(lambda x : golden_dict[x])
start = golden_dict['17:40']
end = golden_dict["18:05"]

train_golden_df = train_df[(train_df["순서"] > 894) & (train_df["순서"] < 919)]

features = ['내부온도관측치', '외부온도관측치', '내부습도관측치', '외부습도관측치', 'CO2관측치', 'EC관측치',
       '최근분무량', '화이트 LED동작강도', '레드 LED동작강도', '블루 LED동작강도', '냉방온도', '냉방부하',
       '난방온도', '난방부하', '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량']

img2data = {}
for i in tqdm(train_golden_df["이미지"].unique()):
       temp = train_golden_df[train_golden_df["이미지"]==i]
       temp_mean = temp[features].mean(axis=0)
       img2data[temp["이미지"].values[0]] = temp_mean["EC관측치"]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1592/1592 [00:07<00:00, 202.18it/s]


In [7]:


class PYTORCH_DATASET(nn.Module):
    def __init__(self, path: list, img2weight: dict, imgs: np.array, img2data: dict, \
                 transform=None):
        super(PYTORCH_DATASET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform
        self.img2weight = img2weight
        self.img2data = img2data

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = self.img2data[img_path]
        df_label = self.img2weight[img_path]

        after_img = self.transform(image=before_img)["image"]
        return {"사진": after_img, "정보": df_index, "상표": df_label}

In [11]:

from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

image_size = 224
train_mode = albu.Compose([
    albu.RandomResizedCrop(height=image_size, width=image_size,
                           scale=(0.25, 1.0), ratio=(0.75, 1.3333333333333333),
                           interpolation=1, p=1.0),
    albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
                          rotate_limit=30, interpolation=1, border_mode=0, value=0, p=0.25),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    ToTensorV2(),
])

test_mode = albu.Compose([
    #                 albu.Resize(self.image_size, self.image_size),
    albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    ToTensorV2(),
])



In [15]:

class CNN_RNN(nn.Module):
    def __init__(self):
        super(CNN_RNN, self).__init__()
        self.model = timm.create_model("efficientnet_b0", pretrained=True, \
                                       num_classes=1)
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        n_class = 1281
        n_hidden = 1024
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
        self.linear = nn.Linear(1024, 256)
        self.output = nn.Linear(256, 1)
        self.relu = nn.ReLU()

    def forward(self, hidden, cell, x, x2):
        hidden_cell = (hidden, cell)
        x = self.model.forward_features(x)
        x = self.avg(x)
        x = x.flatten()
        x = x.view(1, -1, 1280)
        x2 = x2.view(1, -1, 1)
        x = torch.cat([x,x2], dim=-1)
        x, _ = self.lstm(x, hidden_cell)
        x = x[-1]

        x = self.linear(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [16]:

class LIGHTNING_MODEL(pl.LightningModule):
    def __init__(self):
        super(LIGHTNING_MODEL, self).__init__()
        self.model = CNN_RNN()
        self.train_nmae = NMAE()
        self.valid_nmae = NMAE()

    def forward(self, x):
        output = self.model(x)
        return output

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        img = train_batch["사진"]
        data = train_batch["정보"]
        label = train_batch["상표"]

        batch = label.shape[0]
        hidden = torch.zeros(1, batch, 1024, requires_grad=True,device="cuda")
        cell = torch.zeros(1, batch, 1024, requires_grad=True,device="cuda")

        pred = self.model(hidden, cell, img, data)
        pred = pred.view(-1,1)
        label = label.view(-1,1)
        loss = nn.L1Loss()(pred, label)
        self.log("train_loss", loss)
        train_nmae = self.train_nmae(pred, label)
        self.log('train_NMAE', train_nmae)
        return loss

    def training_epoch_end(self, outputs):
        self.train_nmae.reset()

    def validation_step(self, val_batch, batch_idx):
        img = val_batch["사진"]
        data = val_batch["정보"]
        label = val_batch["상표"]

        batch = label.shape[0]
        hidden = torch.zeros(1, batch, 1024, requires_grad=True, device="cuda")
        cell = torch.zeros(1, batch, 1024, requires_grad=True, device="cuda")

        pred = self.model(hidden, cell, img, data)
        pred = pred.view(-1, 1)
        label = label.view(-1, 1)
        loss = nn.L1Loss()(pred, label)
        self.log("val_loss", loss)
        self.valid_nmae.update(pred, label)
        self.log('valid_NMAE', self.valid_nmae, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_epoch_end(self, outputs):
        self.valid_nmae.reset()

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        img = batch["사진"]
        pred = self.model(img)
        return pred


In [ ]:

kf = KFold(n_splits=15, shuffle=True, random_state=2022)
if __name__ == '__main__':
    for fold, (ti, vi) in enumerate(kf.split(train_path)):
        if fold < 2:
            trainset_path, validset_path = np.array(train_path)[ti], np.array(train_path)[vi]
            train_imgset, valid_imgset = np.array(train_imgs)[ti], np.array(train_imgs)[vi]

            train_dataset = PYTORCH_DATASET(path=trainset_path, img2weight=img2weight, imgs=train_imgset, img2data=img2data,\
                                            transform=train_mode)

            valid_dataset = PYTORCH_DATASET(path=validset_path, img2weight=img2weight, imgs=valid_imgset, img2data=img2data,\
                                            transform=test_mode)


            train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32, num_workers=0)
            valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=32, num_workers=0)

            model = LIGHTNING_MODEL()
            trainer = pl.Trainer(
                devices=[0],
                accelerator='gpu',
                max_epochs=40,
                log_every_n_steps=32,
                precision=16)

            trainer.fit(model, train_loader, valid_loader)


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type    | Params
---------------------------------------
0 | model      | CNN_RNN | 13.7 M
1 | train_nmae | NMAE    | 0     
2 | valid_nmae | NMAE    | 0     
---------------------------------------
13.7 M    Trainable params
0         Non-trainable params
13.7 M    Total params
27.442    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

D:\meta2\venv\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: The ``compute`` method of metric NMAE was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
D:\meta2\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

In [25]:
#epoch7 0.286
img2weight


{'CASE01_01': 49.1875,
 'CASE01_02': 59.75,
 'CASE01_03': 72.1875,
 'CASE01_04': 85.75,
 'CASE01_05': 102.5625,
 'CASE01_06': 123.375,
 'CASE01_07': 156.125,
 'CASE01_08': 174.875,
 'CASE01_09': 187.5,
 'CASE02_01': 194.625,
 'CASE02_02': 198.375,
 'CASE02_03': 196.75,
 'CASE02_04': 197.375,
 'CASE02_05': 200.25,
 'CASE02_06': 201.5,
 'CASE02_07': 197.375,
 'CASE02_08': 186.0,
 'CASE02_09': 165.75,
 'CASE02_10': 154.0,
 'CASE02_11': 142.75,
 'CASE03_01': 11.3359375,
 'CASE03_02': 15.7265625,
 'CASE03_03': 21.484375,
 'CASE03_04': 27.09375,
 'CASE03_05': 32.28125,
 'CASE04_01': 57.15625,
 'CASE04_02': 73.875,
 'CASE04_03': 90.75,
 'CASE04_04': 110.5625,
 'CASE04_05': 129.5,
 'CASE04_06': 156.875,
 'CASE04_07': 194.0,
 'CASE04_08': 225.75,
 'CASE04_09': 247.875,
 'CASE04_10': 269.75,
 'CASE04_11': 280.0,
 'CASE04_12': 283.75,
 'CASE04_13': 286.25,
 'CASE04_14': 295.75,
 'CASE04_15': 289.75,
 'CASE04_16': 287.75,
 'CASE04_17': 297.75,
 'CASE04_18': 304.25,
 'CASE04_19': 307.75,
 'CASE04_2